In [1]:
import torch
import torch.utils.data as data
from torchvision import transforms
import pytorch_lightning as pl
from libs.code import *

# custom libraries
from libs.Dataset import *
from libs.VAE import *

In [2]:
# Setting the seed
pl.seed_everything(42)

# do seeed.... np

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# print("Device:", device)

Global seed set to 42


In [3]:
PATH_DST = 'dataset/all_labels.csv'
PATH_GDRIVE = ''
NUM_WORKERS = 4
BATCH_SIZE = 32
NUM_EPOCHS = 5
GPUS = 0

In [4]:
transform = transforms.Compose([
                                transforms.Resize((32,32)),
                                transforms.ToTensor(),
                                # torch.flatten # trasforma il tensore ad una dimensione
                                ])

dataset = TrashbinDataset(csv=PATH_DST, transform=transform)

# TODO: fixa la funzione!
dataset_train, dataset_test = split_into_train_and_test(dataset)
_, dataset_val = split_into_train_and_test(dataset)

train_loader = data.DataLoader(dataset_train, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=True)
val_loader = data.DataLoader(dataset_val, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)
test_loader = data.DataLoader(dataset_test, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
model_dict = {}
# for latent_dim in [64, 128, 256, 384]:
for latent_dim in [384]:
    model_ld, result_ld = train_autoencoder(dataset_train = dataset_train, train_loader=train_loader, val_loader=val_loader, test_loader=test_loader, latent_dim=latent_dim, save_dir='logs_VAE', checkpoint_path='logs_VAE/VAE_test__384/version_0/checkpoints/epoch=0-step=329.ckpt', name="VAE_test_", epochs=1, base_channel_size=32)
    model_dict[latent_dim] = {"model": model_ld, "result": result_ld}

******* Testing..... current GPU: 0 **********


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Found pretrained model, loading from: VAE_test__384.ckpt
Testing: 0it [00:00, ?it/s]

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


Testing: 100%|██████████| 83/83 [00:21<00:00,  3.87it/s]
Testing: 0it [00:00, ?it/s]

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


Testing: 100%|██████████| 83/83 [00:21<00:00,  3.80it/s]
